In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

import joblib

2024-02-26 03:07:20.762648: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 03:07:20.765385: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 03:07:20.794950: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 03:07:20.795007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 03:07:20.795975: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_train.csv', low_memory=False)
x_val = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_val.csv', low_memory=False)
y_val = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_val.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_test.csv', low_memory=False)

In [4]:
x_train = x_train.values
y_train = y_train.values
x_val = x_val.values
y_val = y_val.values
x_test = x_test.values
y_test = y_test.values

In [5]:
model = joblib.load("/home/jovyan/Edge-IIoT/dl/dl_new/dnn.joblib")

2024-02-26 03:07:27.041194: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://4fb0420b225d4206b0e7eb588270bfa6: INVALID_ARGUMENT: ram://4fb0420b225d4206b0e7eb588270bfa6 is a directory.


In [6]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=15,
    input_shape=(48,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [7]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.01)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [8]:
calculate_performance_metrics(adv_test_examples, y_test, model)

952/952 [==============================] - 1s 617us/step
Accuracy: 0.2704993429697766
AUC: 0.5555139898412941

macro
Precision: 0.7033848559449623
Recall: 0.16727447457691766
F1 Score: 0.1475918633790411

weighted
Precision: 0.6295950074705113
Recall: 0.2704993429697766
F1 Score: 0.1873969884629956

Mean FNR: 0.8327255254230823
Mean TNR: 0.9428654000976225
Mean FPR: 0.05713459990237743
Mean TPR: 0.16727447457691766


In [9]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.05)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [10]:
calculate_performance_metrics(adv_test_examples, y_test, model)

952/952 [==============================] - 1s 612us/step
Accuracy: 0.2704664914586071
AUC: 0.5555187731258248

macro
Precision: 0.6367181892782956
Recall: 0.1663612325677852
F1 Score: 0.14645426013765103

weighted
Precision: 0.6232218143036257
Recall: 0.2704664914586071
F1 Score: 0.18735606613877345

Mean FNR: 0.8336387674322148
Mean TNR: 0.9428631959494157
Mean FPR: 0.05713680405058435
Mean TPR: 0.1663612325677852


In [24]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.1)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [26]:
adv_test_examples

array([[3.54400e+03, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [3.25510e+04, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [6.87330e+04, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       ...,
       [1.48579e+05, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [1.48917e+05, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [1.57514e+05, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00]], dtype=float32)

In [25]:
calculate_performance_metrics(adv_test_examples, y_test, model)

952/952 [==============================] - 1s 603us/step
Accuracy: 0.2704993429697766
AUC: 0.5555211237415578

macro
Precision: 0.6367181892782956
Recall: 0.16727447457691766
F1 Score: 0.1475918633790411

weighted
Precision: 0.6232218143036257
Recall: 0.2704993429697766
F1 Score: 0.1873969884629956

Mean FNR: 0.8327255254230823
Mean TNR: 0.9428655366911969
Mean FPR: 0.05713446330880328
Mean TPR: 0.16727447457691766


In [27]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.15)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [29]:
adv_test_examples

array([[3.54400e+03, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [3.25510e+04, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [6.87330e+04, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       ...,
       [1.48579e+05, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [1.48917e+05, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [1.57514e+05, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00]], dtype=float32)

In [28]:
calculate_performance_metrics(adv_test_examples, y_test, model)

952/952 [==============================] - 1s 606us/step
Accuracy: 0.2704993429697766
AUC: 0.5555219177654196

macro
Precision: 0.6367181892782956
Recall: 0.16727447457691766
F1 Score: 0.1475918633790411

weighted
Precision: 0.6232218143036257
Recall: 0.2704993429697766
F1 Score: 0.1873969884629956

Mean FNR: 0.8327255254230823
Mean TNR: 0.9428655366911969
Mean FPR: 0.05713446330880328
Mean TPR: 0.16727447457691766


In [11]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.2)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [12]:
calculate_performance_metrics(adv_test_examples, y_test, model)

952/952 [==============================] - 1s 595us/step
Accuracy: 0.2704993429697766
AUC: 0.5555219423681604

macro
Precision: 0.6367181892782956
Recall: 0.16727447457691766
F1 Score: 0.1475918633790411

weighted
Precision: 0.6232218143036257
Recall: 0.2704993429697766
F1 Score: 0.1873969884629956

Mean FNR: 0.8327255254230823
Mean TNR: 0.9428655366911969
Mean FPR: 0.05713446330880328
Mean TPR: 0.16727447457691766


In [13]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.25)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [14]:
calculate_performance_metrics(adv_test_examples, y_test, model)

952/952 [==============================] - 1s 593us/step
Accuracy: 0.2704993429697766
AUC: 0.5555242887254095

macro
Precision: 0.6367181892782956
Recall: 0.16727447457691766
F1 Score: 0.1475918633790411

weighted
Precision: 0.6232218143036257
Recall: 0.2704993429697766
F1 Score: 0.1873969884629956

Mean FNR: 0.8327255254230823
Mean TNR: 0.9428655366911969
Mean FPR: 0.05713446330880328
Mean TPR: 0.16727447457691766


In [15]:
# Initialize PGD attack
attack = ProjectedGradientDescent(estimator=classifier, eps=0.3)

# Generate adversarial test examples
adv_test_examples = attack.generate(x=x_test)

In [17]:
calculate_performance_metrics(adv_test_examples, y_test, model)

952/952 [==============================] - 1s 606us/step
Accuracy: 0.2704993429697766
AUC: 0.5555242887254095

macro
Precision: 0.6367181892782956
Recall: 0.16727447457691766
F1 Score: 0.1475918633790411

weighted
Precision: 0.6232218143036257
Recall: 0.2704993429697766
F1 Score: 0.1873969884629956

Mean FNR: 0.8327255254230823
Mean TNR: 0.9428655366911969
Mean FPR: 0.05713446330880328
Mean TPR: 0.16727447457691766
